In [ ]:
# Load our libaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import datetime, os
import cv2

In [1]:
!conda update anaconda-navigator  

^C


In [ ]:
def get_dataset():
    #loading dataframes
    train_df = pd.read_csv('./input/boneage-training-dataset.csv')
    test_df = pd.read_csv('./input/boneage-test-dataset.csv')

    #appending file extension to id column for both training and testing dataframes
    train_df['id'] = train_df['id'].apply(lambda x: str(x)+'.png')
    test_df['Case ID'] = test_df['Case ID'].apply(lambda x: str(x)+'.png') 
    train_df['gender'] = train_df['male'].apply(lambda x: 'male' if x else 'female')
    return train_df, test_df

In [ ]:
# Get our Training and Test Data
train_df,_ = get_dataset()

In [ ]:
df = pd.read_csv('./input/boneage-training-dataset.csv')

In [ ]:
belongs_to = np.ones(len(df['boneage']))
for k, age in tqdm(enumerate(df['boneage'])):
    if age <= 3:
        belongs_to[k] = 1
    elif age > 3  and  age <= 6:
        belongs_to[k] = 2
    elif age > 6  and  age <= 9:
        belongs_to[k] = 3
    elif age > 9  and  age <= 12:
        belongs_to[k] = 4
    elif age > 12  and  age <= 15:
        belongs_to[k] = 5
    elif age > 15  and  age <= 18:
        belongs_to[k] = 6
    elif age > 18  and  age <= 21:
        belongs_to[k] = 7
    elif age > 21  and  age <= 24:
        belongs_to[k] = 8
    elif age > 24  and  age <= 30:
        belongs_to[k] = 9
    elif age > 30  and  age <= 36:
        belongs_to[k] = 10
    elif age > 36  and  age <= 42:
        belongs_to[k] = 11
    elif age > 42  and  age <= 48:
        belongs_to[k] = 12
    elif age > 48  and  age <= 54:
        belongs_to[k] = 13
    elif age > 54  and  age <= 60:
        belongs_to[k] = 14
    elif age > 60  and  age <= 66:
        belongs_to[k] = 15
    elif age > 66  and  age <= 72:
        belongs_to[k] = 16
    elif age > 72  and  age <= 78:
        belongs_to[k] = 17
    elif age > 78  and  age <= 84:
        belongs_to[k] = 18
    elif age > 84  and  age <= 90:
        belongs_to[k] = 19
    elif age > 90  and  age <= 96:
        belongs_to[k] = 20
    elif age > 96  and  age <= 102:
        belongs_to[k] = 21
    elif age > 102  and  age <= 108:
        belongs_to[k] = 22
    elif age > 108  and  age <= 114:
        belongs_to[k] = 23
    elif age > 114  and  age <= 120:
        belongs_to[k] = 24
    elif age > 120  and  age <= 132:
        belongs_to[k] = 25
    elif age > 132  and  age <= 144:
        belongs_to[k] = 26
    elif age > 144  and  age <= 156:
        belongs_to[k] = 27
    elif age > 156  and  age <= 168:
        belongs_to[k] = 28
    elif age > 168  and  age <= 180:
        belongs_to[k] = 29
    elif age > 180  and  age <= 192:
        belongs_to[k] = 30
    elif age > 192  and  age <= 204:
        belongs_to[k] = 31
    elif age > 204  and  age <= 216:
        belongs_to[k] = 32

In [ ]:
df['belongs_to_group'] = belongs_to.tolist()

In [ ]:
df['belongs_to_group'] = pd.to_numeric(df['belongs_to_group'], downcast='integer')

In [ ]:
DIR = './new_dataset'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))
length = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])

In [ ]:
y = df['belongs_to_group']

In [ ]:
data_size = (128 * 128)
data = np.ones(length * data_size)
data = data.reshape((length, 128 , 128))
new_y = np.ones(length)
count = 0
cnt = 0

for filename, boneage in tqdm(list(train_df[['id','boneage']].values)):
    try:
        fname = './new_dataset/'+ filename
        img = cv2.imread(fname)
        img  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
        new_y[count] = y[cnt]
        data[count,:,:] = img.tolist()
        count += 1
    except Exception as ex:
        continue
    cnt += 1

In [ ]:
images=data.reshape(length,-1)

In [ ]:
count=0
cnt=-1
for filename, boneage in tqdm(list(train_df[['id','boneage']].values)):
    cnt += 1
    fname = './new_dataset/'+ filename
    if os.path.isfile(fname):
        new_y[count] = y[cnt]
        count += 1        
    else:
        continue

In [ ]:
# Get our training labels
labels = new_y

In [ ]:
# View the unique labels, 32 in total
unique_val = np.array(labels)
np.unique(unique_val)

In [ ]:
# hot one encode our labels
from sklearn.preprocessing import LabelBinarizer

label_binrizer = LabelBinarizer()
labels = label_binrizer.fit_transform(labels)

In [ ]:
# View our labels
labels

In [ ]:
images

In [ ]:
# Inspect an image
index = 13
print(labels[index])
plt.imshow(images[index].reshape(128,128))
plt.show()

In [ ]:
# Split our data into x_train, x_test, y_train and y_test
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = 101)

In [ ]:
# Start loading our tensorFlow modules and define our batch size etc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

batch_size = 64
num_classes = 32
epochs = 50

In [ ]:
# Scale our images
x_train = x_train / 255
x_test = x_test / 255

In [ ]:
# Reshape them into the size required by TF and Keras
x_train = x_train.reshape(x_train.shape[0], 128, 128, 1)
x_test = x_test.reshape(x_test.shape[0], 128, 128, 1)

plt.imshow(x_train[0].reshape(128,128))

In [ ]:
plt.show()

In [ ]:
# Create our CNN Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam 

model = Sequential()
model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu', input_shape=(128, 128 ,1) ))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(256, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(512, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(1024, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.20))

model.add(Dense(num_classes, activation = 'softmax'))

In [ ]:
# Compile our Model
model.compile(loss = 'categorical_crossentropy',
              optimizer= Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
# Train our Model
history = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs=epochs, batch_size=batch_size)

In [ ]:
# Save our Model
model.save("sign_mnist_cnn_50_Epochs.h5")
print("Model Saved")

In [ ]:
# View our training history graphically
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Accuracy")
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train','test'])

plt.show()

In [ ]:
# Reshape our test data so that we can evaluate it's performance on unseen data
test_labels = test['label']
test.drop('label', axis = 1, inplace = True)

test_images = test.values
test_images = np.array([np.reshape(i, (28, 28)) for i in test_images])
test_images = np.array([i.flatten() for i in test_images])

test_labels = label_binrizer.fit_transform(test_labels)

test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

test_images.shape

y_pred = model.predict(test_images)

In [ ]:
# Get our accuracy score
from sklearn.metrics import accuracy_score

accuracy_score(test_labels, y_pred.round())

In [ ]:
# Create function to match label to letter
def getLetter(result):
    classLabels = { 0: 'A',
                    1: 'B',
                    2: 'C',
                    3: 'D',
                    4: 'E',
                    5: 'F',
                    6: 'G',
                    7: 'H',
                    8: 'I',
                    9: 'K',
                    10: 'L',
                    11: 'M',
                    12: 'N',
                    13: 'O',
                    14: 'P',
                    15: 'Q',
                    16: 'R',
                    17: 'S',
                    18: 'T',
                    19: 'U',
                    20: 'V',
                    21: 'W',
                    22: 'X',
                    23: 'Y'}
    try:
        res = int(result)
        return classLabels[res]
    except:
        return "Error"

# Test on Actual Webcam Input

In [ ]:
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    ##############################
    frame=cv2.flip(frame, 1)

    #define region of interest
    roi = frame[100:400, 320:620]
    cv2.imshow('roi', roi)
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    roi = cv2.resize(roi, (28, 28), interpolation = cv2.INTER_AREA)
    
    cv2.imshow('roi sacled and gray', roi)
    copy = frame.copy()
    cv2.rectangle(copy, (320, 100), (620, 400), (255,0,0), 5)
    
    roi = roi.reshape(1,28,28,1) 

    result = str(model.predict_classes(roi, 1, verbose = 0)[0])
    cv2.putText(copy, getLetter(result), (300 , 100), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 255, 0), 2)
    cv2.imshow('frame', copy)    
    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows() 

## Performance is less than adequate because our training data is very different from our real life data

Let's create our own dataset using your webcam!

In [ ]:
# Create a function to setup the directories we'll be storing our images
def makedir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        return None
    else:
        pass

## Let's use OpenCV to Build our Dataset

In [ ]:
import cv2
import os

cap = cv2.VideoCapture(0)

i=0
image_count = 0
    
while i < 7:

    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    #define region of interest
    roi = frame[100:400, 320:620]
    cv2.imshow('roi', roi)
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    roi = cv2.resize(roi, (28, 28), interpolation = cv2.INTER_AREA)

    cv2.imshow('roi sacled and gray', roi)
    copy = frame.copy()
    cv2.rectangle(copy, (320, 100), (620, 400), (255,0,0), 5)
    
    if i == 0:
        image_count = 0
        cv2.putText(copy, "Hit Enter to Record when ready", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
    if i == 1:
        image_count+=1
        cv2.putText(copy, "Recording 1st gesture - Train", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.putText(copy, str(image_count), (400 , 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        gesture_one = './handgestures/train/0/'
        makedir(gesture_one)
        cv2.imwrite(gesture_one + str(image_count) + ".jpg", roi)
    if i == 2:
        image_count+=1
        cv2.putText(copy, "Recording 1st gesture - Test", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.putText(copy, str(image_count), (400 , 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        gesture_one = './handgestures/test/0/'
        makedir(gesture_one)
        cv2.imwrite(gesture_one + str(image_count) + ".jpg", roi)
    if i == 3:
        cv2.putText(copy, "Hit Enter to Record when ready to Record 2nd gesture", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
    if i == 4:
        image_count+=1
        cv2.putText(copy, "Recording 2nd gesture - Train", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.putText(copy, str(image_count), (400 , 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        gesture_two = './handgestures/train/1/'
        makedir(gesture_two)
        cv2.imwrite(gesture_two + str(image_count) + ".jpg", roi)
    if i == 5:
        image_count+=1
        cv2.putText(copy, "Recording 2nd gesture - Test", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.putText(copy, str(image_count), (400 , 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        gesture_two = './handgestures/test/1/'
        makedir(gesture_two)
        cv2.imwrite(gesture_two + str(image_count) + ".jpg", roi)
    if i == 6:
        cv2.putText(copy, "Hit Enter to Exit", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
    cv2.imshow('frame', copy)    

    if cv2.waitKey(1) == 13: #13 is the Enter Key
        image_count = 0
        i+=1

cap.release()
cv2.destroyAllWindows() 

## Let's use Keras's Data Augmentation to enhance our dataset

In [ ]:
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os

num_classes = 2
img_rows, img_cols = 28, 28
batch_size = 32

train_data_dir = './handgestures/train'
validation_data_dir = './handgestures/test'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        color_mode = 'grayscale',
        class_mode='binary')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        color_mode = 'grayscale',
        class_mode='binary')

### Creating our Model

In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu', input_shape=(28,28,1) ))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.20))

model.add(Dense(1, activation = 'sigmoid'))

print(model.summary())

### Training our Model

In [ ]:
# We use a very small learning rate 
model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

nb_train_samples = 1206 
nb_validation_samples = 301 
epochs = 10

history = model.fit(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

In [ ]:
# Save our Model
model.save("my_gestures_cnn.h5")

In [ ]:
from tensorflow.keras.models import load_model

classifier = load_model('my_gestures_cnn.h5')

## Now let's test the model we just built!

In [ ]:
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    ##############################
    frame=cv2.flip(frame, 1)

    #define region of interest
    roi = frame[100:400, 320:620]
    cv2.imshow('roi', roi)
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    roi = cv2.resize(roi, (28, 28), interpolation = cv2.INTER_AREA)
    
    cv2.imshow('roi scaled and gray', roi)
    copy = frame.copy()
    cv2.rectangle(copy, (320, 100), (620, 400), (255,0,0), 5)
    
    roi = roi.reshape(1,28,28,1) 
    roi = roi/255
    result = str(classifier.predict_classes(roi, 1, verbose = 0)[0])
    cv2.putText(copy, str(result), (300 , 100), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 255, 0), 2)
    cv2.imshow('frame', copy)    
    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows() 

In [ ]:
cap.release()
cv2.destroyAllWindows() 